<a href="https://colab.research.google.com/github/GokulVijayaraghavan/Understanding-Indirect-Answers/blob/main/Boolq_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install datasets transformers
!pip install sklearn

     |████████████████████████████████| 298 kB 5.4 MB/s 
     |████████████████████████████████| 3.1 MB 37.5 MB/s 
     |████████████████████████████████| 1.1 MB 26.3 MB/s 
     |████████████████████████████████| 243 kB 48.5 MB/s 
     |████████████████████████████████| 59 kB 6.2 MB/s 
     |████████████████████████████████| 132 kB 44.2 MB/s 
     |████████████████████████████████| 895 kB 37.5 MB/s 
     |████████████████████████████████| 3.3 MB 18.1 MB/s 
     |████████████████████████████████| 596 kB 39.9 MB/s 
     |████████████████████████████████| 160 kB 49.3 MB/s 
     |████████████████████████████████| 192 kB 51.3 MB/s 
     |████████████████████████████████| 271 kB 45.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from datasets import load_dataset, load_metric
from transformers import DistilBertTokenizerFast
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import Trainer, TrainingArguments

In [ ]:
checkpoint = "distilbert-base-uncased"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_excel('/content/drive/MyDrive/circa_validate.xlsx')
df.head(5)

,Unnamed: 0,id,context,question-X,canquestion-X,answer-Y,judgements,goldstandard1,goldstandard2
0,4056,4056,Y has just told X that he/she is considering s...,Is your work technical?,My work is technical .,it can be at times,Probably yes / sometimes yes#Probably yes / so...,Probably yes / sometimes yes,Yes
1,19824,19824,Y has just travelled from a different city to ...,Are you planning to drink?,I am planning to drink .,I'm two months sober.,No#No#No#No#No,No,No
2,19383,19383,Y has just told X that he/she is considering s...,Do you make enough money at your job?,I make enough money at my job .,I could do with some more,"Yes, subject to some conditions#No#No#Probably...",NaN,No
3,28619,28619,Y has just moved into a neighbourhood and meet...,Do you feel like having a warm meal?,I feel like having a warm meal .,It's so cold out; that sounds tempting.,Yes#Probably yes / sometimes yes#Probably yes ...,Probably yes / sometimes yes,Yes
4,9837,9837,Y has just travelled from a different city to ...,Did you know there's no sales tax in Oregon?,I did know there 's no sales tax in Oregon .,I had never heard that before,No#No#No#No#Probably yes / sometimes yes,No,No


In [ ]:
dataset = load_dataset("boolq")
dataset

Using custom data configuration default
Reusing dataset boolq (/root/.cache/huggingface/datasets/boolq/default/0.1.0/bf0dd57da941c50de94ae3ce3cef7fea48c08f337a4b7aac484e9dddc5aa24e5)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 9427
    })
    validation: Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 3270
    })
})

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained(checkpoint)


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(example):
  encoded = tokenizer(example["question"], example["passage"], truncation=True)
  encoded["labels"] = [int(a) for a in example["answer"]]
  return encoded

tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classi

In [ ]:
train_dataset=tokenized_datasets["train"]
print(train_dataset)

Dataset({
    features: ['answer', 'attention_mask', 'input_ids', 'labels', 'passage', 'question'],
    num_rows: 9427
})


In [ ]:
args = TrainingArguments("roberta-booql", per_device_train_batch_size=16, learning_rate=1e-3,num_train_epochs=3)

In [ ]:
trainer = Trainer(model, args, train_dataset=tokenized_datasets["train"], eval_dataset=tokenized_datasets["df"], data_collator=data_collator, tokenizer=tokenizer,)

KeyError: ignored

In [ ]:
trainer.train()

In [ ]:
trainer = Trainer(model, args, train_dataset=tokenized_datasets["train"], eval_dataset=tokenized_datasets["df"], data_collator=data_collator, tokenizer=tokenizer,)

In [ ]:
trainer = Trainer(model, args, train_dataset=tokenized_datasets["train"], eval_dataset=tokenized_datasets["df"], data_collator=data_collator, tokenizer=tokenizer,)

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
y_pred = predictions.predictions.argmax(-1)
labels = predictions.label_ids

In [ ]:
metric = load_metric("accuracy")
metric.compute(predictions=y_pred, references=predictions.label_ids)